In [1]:
import matplotlib.pyplot as plt
import random as r
import pprint as pp
import numpy as np
import csv
import tensorflow as tf
from tensorflow import keras

In [2]:
# Define type of slice and its parameters
slices_type = {
    'eMBB' : {
        'bandwidth_max': 100000000, #max bandwidth which BS can give to this slice
        'client_weight': 0.3, # Portion of clients which use this type of slice
        'band_guaranteed' : 0, # based on quality of service
        'min': 4000000, 
        'max': 500000000 # how much bps can require this type of slice
      },
      'URLLC' : {
        'bandwidth_max': 10000000,
        'client_weight': 0.2,
        'band_guaranteed' : 5000000,
        'min': 5000000,
        'max': 8000000
      },
      'MIoT' : {
        'bandwidth_max': 10000000,
        'client_weight': 0.15,
        'band_guaranteed' : 1000000,
        'min': 1000000, 
        'max': 8000000
      },
      'mMTC' : {
        'bandwidth_max': 10000000,
        'client_weight': 0.15,
        'band_guaranteed' : 1000000,
        'min': 1000000, 
        'max': 8000000
      },
      'voice' : {
        'bandwidth_max': 1000000,
        'client_weight': 0.2,
        'band_guaranteed' : 500000,
        'min': 4000000, 
        'max': 8000000
     }
}
          
# Define base station parameters
basestation = {
    'capacity': 20000000000,   #20Gbps
    'coverage': 250,
    'ratios': { # portion of capacity dedicated to various slice type
        'URLLC': 0.05,
        'eMBB': 0.55,
        'mMTC': 0.15,
        'voice': 0.1,
        'MIoT': 0.15
    },
    'x': 300,
    'y': 300
}

# Define mobility patterns and its parameters
mobility_patterns = {
  'car': {
    'distribution': 'normal',
    'params':(0,7),
    'client_weight': 0.10
  },
  'walk': {
    'distribution': 'randint',
    'params': (-1, 1),
    'client_weight': 0.40
  },
  'stationary': {
    'distribution': 'normal',
    'params': (0, 0.1),
    'client_weight': 0.20
  },
  'publictransport': {
    'distribution': 'randint',
    'params': (-4, 4),
    'client_weight': 0.10
  },
  'slackperson':{
    'distribution': 'randint',
    'params':(0, 1),
    'client_weight': 0.20
  }
}

In [3]:
# Define obj Client
class Client:
    def __init__(self, ID, x, y, mobility_pattern, freq, slice_type, bs, slice_cap = None):
        self.ID = ID
        self.x = x
        self.y = y
        self.mobility_pattern = mobility_pattern
        self.freq = freq
        self.slice_type = slice_type
        self.bs = bs
        self.usage_rem = 0
        self.last_usage = 0
        self.connected = False
        
    def step_1(self):
        if self.bs is not None:
            if self.usage_rem > 0:
                if self.connected:
                    self.start_consume()
                else:
                    self.connect()
            else:
                if self.connected:
                    self.disconnect()
                else:
                    self.generate_usage_and_connect()
        
    def step_2(self):
        if self.connected and self.last_usage > 0:
            self.release_consume()
            if self.usage_rem == 0:
                self.disconnect()
    
    def step_3(self):
        x, y = self.move()
        self.x += x
        self.y += y
        if self.bs is not None:
            if not inRange(self, bs):
                self.disconnect()       
                
    def generate_usage_and_connect(self):
        if self.slice_type is not None:
            # Generate new usage
            self.usage_rem = self.generate()
            self.connect()
            print(f"Client_{self.ID}, coordinates : ({self.x}, {self.y}) requests {self.usage_rem} usage to BS{self.bs.ID} to slice {self.slice_type}")     

    def connect(self):
        s = self.slice_type
        
        if self.connected:
            return
        
        if self.is_avaliable():         # threshold for usage
            self.connected = True
            bs.connected_users += 1
            
            if s == 'eMBB':
                bs.embb_users += 1
            if s == 'URLLC':
                bs.urllc_users += 1
            if s == 'MIoT':
                bs.miot_users += 1
            if s == 'mMTC':
                bs.mmtc_users += 1
            if s == 'voice':
                bs.voice_users += 1
                
            print(f"Client {self.ID}, coordinates : ({self.x}, {self.y}) connected to slice {s} @ BS{self.bs.ID}")
        else:
            self.connected = False
            print(f"Client {self.ID}, coordinates : ({self.x}, {self.y}) connection refused to slice {s} @ BS{self.bs.ID}")
        
    def disconnect(self):
        s = self.slice_type
        
        if self.connected == True:
            
            bs.connected_users -= 1
            
            if s == 'eMBB':
                bs.embb_users -= 1
            if s == 'URLLC':
                bs.urllc_users -= 1
            if s == 'MIoT':
                bs.miot_users -= 1
            if s == 'mMTC':
                bs.mmtc_users -= 1
            if s == 'voice':
                bs.voice_users -= 1
                
            self.connected = False
            
        print(f"Client {self.ID}, coordinates: ({self.x},{self.y}) disconnected from slice {self.slice_type} @ BS{self.bs.ID} ")
    
    def start_consume(self):
        amount = min(self.usage_rem, self.get_consumable_share())    # bandwidth allocation
        self.last_usage = amount
        print(f"Client {self.ID}, coordinates: ({self.x}, {self.y}) gets {amount} usage from @ BS{self.bs.ID} for slice {self.slice_type}")
           
    def release_consume(self):
        # release resources
        if self.last_usage > 0:
            self.usage_rem -= self.last_usage
            print(f"Client {self.ID}, coordinates: ({self.x}, {self.y}), releases {self.last_usage} usage to BS{self.bs.ID}")
            self.last_usage = 0
        
    # update client coordinates    
    def move(self):   
        distr = self.mobility_pattern.distribution
        params = self.mobility_pattern.params
        if distr == "randint":
            delta_x = r.randint(params[0], params[1])
            delta_y = r.randint(params[0], params[1])
            return delta_x, delta_y
        if distr == "normal":
            delta_x = np.random.normal(params[0], params[1])
            delta_y = np.random.normal(params[0], params[1])
            return delta_x, delta_y 
        
    # generate client consume   
    def generate(self):
        s = self.slice_type
        cicciolina = r.randint(slices_type[s]['min'], slices_type[s]['max'])
        return cicciolina
    
    def is_avaliable(self):
        global capacity
        s = self.slice_type
        
        if s == 'eMBB':
            bandwidth_next = capacity[s] / (bs.embb_users + 1)
        if s == 'URLLC':
            bandwidth_next = capacity[s] / (bs.urllc_users + 1)
        if s == 'MIoT':
            bandwidth_next = capacity[s] / (bs.miot_users + 1)
        if s == 'mMTC':
            bandwidth_next = capacity[s] / (bs.mmtc_users + 1)
        if s == 'voice':
            bandwidth_next = capacity[s] / (bs.voice_users + 1)
       
        if bandwidth_next < slices_type[s]['band_guaranteed']:
            return False

        return True
      
    def get_consumable_share(self):
        global capacity
        s = self.slice_type

        if s == 'eMBB':
            if bs.embb_users <= 0:
                return int(min(capacity[s], slices_type[s]['bandwidth_max']))
            else: 
                return int(min(capacity[s]/bs.embb_users, slices_type[s]['bandwidth_max']))
        
        if s == 'URLLC':
            if bs.urllc_users <= 0:
                return int(min(capacity[s], slices_type[s]['bandwidth_max']))
            else:
                return int(min(capacity[s]/bs.urllc_users, slices_type[s]['bandwidth_max']))
        
        if s == 'MIoT':
            if bs.miot_users <= 0:
                return int(min(capacity[s], slices_type[s]['bandwidth_max']))
            else:
                return int(min(capacity[s]/bs.miot_users, slices_type[s]['bandwidth_max']))
        
        if s == 'mMTC':
            if bs.mmtc_users <= 0:
                return int(min(capacity[s], slices_type[s]['bandwidth_max']))
            else: 
                return int(min(capacity[s]/bs.mmtc_users, slices_type[s]['bandwidth_max']))
        
        if s == 'voice':
            if bs.voice_users <= 0:
                return int(min(capacity[s], slices_type[s]['bandwidth_max']))
            else:
                return int(min(capacity[s]/bs.voice_users, slices_type[s]['bandwidth_max']))
        

In [4]:
# Define obj Mobility 
class Mobility:
    def __init__(self, name, distribution, params):
        self.name = name
        self.distribution = distribution
        self.params = params
        

In [5]:
# Define obj BaseStation
class BaseStation:
    def __init__(self,ID, x, y, coverage, capacity, ratios):
        self.ID =ID
        self.x = x
        self.y = y
        self.coverage = coverage
        self.capacity = capacity
        self.ratios = ratios
        self.pot_band_remaining = capacity
        self.band_remaining = capacity
        self.connected_users = 0
        self.embb_users = 0
        self.urllc_users = 0
        self.miot_users = 0
        self.mmtc_users = 0
        self.voice_users = 0
        

In [6]:
def get_random_mobility_pattern(vals, mobility_patterns):
    i = 0
    j = r.random()

    while vals[i] < j:
        i += 1

    return mobility_patterns[i]

def get_random_slice_type(vals):
    i = 0
    j = r.random()
    s = ''

    while vals[i] < j:
        i += 1
    
    if i == 0:
       s = 'eMBB'
    if i == 1:
       s = 'URLLC' 
    if i == 2:
       s = 'MIoT' 
    if i == 3:
       s = 'mMTC' 
    if i == 4:
       s = 'voice' 
    
    return s

def inRange(client, bs):
    x_client = client.x
    y_client = client.y
    x_bs = bs.x
    y_bs = bs.y
    if (abs(x_client - x_bs) < bs.coverage) and (abs(y_client - y_bs) < bs.coverage):
        return True
        print('true')
    else:
        return False
        print('false')
        
def get_new_ratios(prediction):
    band = basestation.get('capacity')
    
    # Compute new cpacity
    embb = int(prediction[0]*band)
    urllc = int(prediction[1]*band)
    miot = int(prediction[2]*band)
    mmtc = int(prediction[3]*band)
    voice = int(prediction[4]*band)

    out = { 'eMBB': embb,
            'URLLC': urllc,
            'MIoT': miot,
            'mMTC': mmtc,
            'voice': voice
          }
    
    return out

collected, slice_weights = 0, []
for key, value in slices_type.items():
    collected += value['client_weight']
    slice_weights.append(collected)

collected, mb_weights = 0, []
for key, value in mobility_patterns.items():
    collected += value['client_weight']
    mb_weights.append(collected)
    
m_patterns = []
for name, mb in mobility_patterns.items():
    mobility_pattern = Mobility(name, mb['distribution'], mb['params'])
    m_patterns.append(mobility_pattern)   

# Load RNN
model = tf.keras.models.load_model('model.h5')


In [7]:
# Define class Request
class Request:
    def __init__(self, id_client, type_slice, traffic_request, traffic_left):
        self.id_client = id_client
        self.type_slice = type_slice
        self.traffic_request = traffic_request
        self.traffic_left = traffic_left
        
#Define class csv
class CsvData:
    def __init__(self, step, r_embb, g_embb, connect_embb, r_urllc, g_urllc, connect_urllc, r_miot, g_miot, connect_miot, r_mmtc, g_mmtc, connect_mmtc, r_voice, g_voice, connect_voice):
        self.step = step
        self.r_embb = r_embb
        self.g_embb = g_embb
        self.connect_embb = connect_embb
        self.r_urllc = r_urllc
        self.g_urllc = g_urllc
        self.connect_urllc = connect_urllc
        self.r_miot = r_miot
        self.g_miot = g_miot
        self.connect_miot = connect_miot
        self.r_mmtc = r_mmtc
        self.g_mmtc = g_mmtc
        self.connect_mmtc = connect_mmtc
        self.r_voice = r_voice
        self.g_voice = g_voice
        self.connect_voice = connect_voice
        

In [8]:
# Round 0: create network area, place base station and clients
num_clients = 1000
simulation_time = 30
x_area = 600
y_area = 600

bs = BaseStation(0, basestation.get('x'), basestation.get('y'), basestation.get('coverage'), basestation.get('capacity'), basestation.get('ratios'))

clients = []
for i in range(num_clients):
    x = r.randint(0, x_area)
    y = r.randint(0, y_area)
    freq = r.random()
    
    mobility_pattern = get_random_mobility_pattern(mb_weights, m_patterns)
    slice_type = get_random_slice_type(slice_weights) 
    
    c = Client(i, x, y, mobility_pattern, freq, slice_type, bs)
    clients.append(c)


In [9]:
clients_inRange = []

for i in range(num_clients):
    if inRange(clients[i], bs):
        clients_inRange.append(clients[i])

# create fixed size list to manage the request
request_list = [None] * len(clients)
active_clients = []
csv_final = []
capacity = { 'eMBB' : 11000000000,
             'URLLC' :1000000000,
             'MIoT' : 3000000000,
             'mMTC' : 3000000000,
             'voice' : 2000000000
           }

for n in range(simulation_time):
    
    # manage number active clients
    active = []
    new_connection = r.randint(0, len(clients_inRange))
    active = r.sample(clients_inRange, new_connection)
    
    for j in range(len(active)):
        if request_list[active[j].ID] == None:
            active_clients.append(active[j])

    print('STEP', n)
    
    for i in range(len(active_clients)):   
        # step 1 and create request
        active_clients[i].step_1()
        
        req = Request(active_clients[i].ID, active_clients[i].slice_type, active_clients[i].usage_rem, active_clients[i].usage_rem)
        request_list[active_clients[i].ID] = req
  
    for i in range(len(active_clients)):
        # step 2 and update request (delete non active user, and update traffic)
        active_clients[i].step_2()
        request_list[active_clients[i].ID].traffic_left = active_clients[i].usage_rem
    
    user_embb, user_urllc, user_miot, user_mmtc, user_voice = 0, 0, 0, 0, 0
    req_embb, req_urllc, req_miot, req_mmtc, req_voice = 0, 0, 0, 0, 0
    giv_embb, giv_urllc, giv_miot, giv_mmtc, giv_voice = 0, 0, 0, 0, 0
    for item in request_list:
     if item is not None:
        if item.type_slice == 'eMBB':
            if item.traffic_request - item.traffic_left != 0:
                req_embb += item.traffic_request
                giv_embb += item.traffic_request - item.traffic_left 
                user_embb += 1
            else:
                req_embb += 0
                giv_embb += 0
                
        if item.type_slice == 'URLLC' :
            if item.traffic_request - item.traffic_left != 0:
                req_urllc += item.traffic_request
                giv_urllc += item.traffic_request - item.traffic_left
                user_urllc += 1
            else:
                req_urllc += 0
                giv_urllc += 0
 
        if item.type_slice == 'MIoT':
            if item.traffic_request - item.traffic_left != 0:
                req_miot += item.traffic_request
                giv_miot += item.traffic_request - item.traffic_left
                user_miot += 1
            else:
                req_miot += 0
                giv_miot += 0
 
        if item.type_slice == 'mMTC':
            if item.traffic_request - item.traffic_left != 0:
                req_mmtc += item.traffic_request
                giv_mmtc += item.traffic_request - item.traffic_left
                user_mmtc += 1
            else:
                req_mmtc += 0
                giv_mmtc += 0

        if item.type_slice == 'voice':
            if item.traffic_request - item.traffic_left != 0:
                req_voice += item.traffic_request
                giv_voice += item.traffic_request - item.traffic_left
                user_voice += 1
            else:
                req_voice += 0
                giv_voice += 0
    
    csv_data = CsvData(n, req_embb, giv_embb, user_embb, req_urllc, giv_urllc, user_urllc, req_miot, giv_miot, user_miot, req_mmtc, giv_mmtc, user_mmtc, req_voice, giv_voice, user_voice)  
    csv_final.append(csv_data)
    # preditc ratios
    new_ratios = [[[req_embb, req_urllc, req_miot, req_mmtc, req_voice]]]
    new_ratios = np.array(new_ratios)/bs.capacity
    prediction = model.predict(new_ratios)
    capacity = get_new_ratios(prediction[0])
    
    for i in range(len(active_clients)):
        if active_clients[i].connected == False:
            request_list[active_clients[i].ID] = None

    for item in active_clients:
        # step 3 and delete also user not in range
        item.step_3()
        if item.connected == False:
            request_list[item.ID] = None
            active_clients.remove(item)
            

STEP 0
Client 549, coordinates : (466, 445) connected to slice eMBB @ BS0
Client_549, coordinates : (466, 445) requests 358079688 usage to BS0 to slice eMBB
Client 12, coordinates : (75, 228) connected to slice URLLC @ BS0
Client_12, coordinates : (75, 228) requests 5876528 usage to BS0 to slice URLLC
Client 468, coordinates : (198, 386) connected to slice eMBB @ BS0
Client_468, coordinates : (198, 386) requests 225752028 usage to BS0 to slice eMBB
Client 822, coordinates : (418, 375) connected to slice eMBB @ BS0
Client_822, coordinates : (418, 375) requests 383250040 usage to BS0 to slice eMBB
Client 783, coordinates : (275, 527) connected to slice MIoT @ BS0
Client_783, coordinates : (275, 527) requests 3458329 usage to BS0 to slice MIoT
Client 791, coordinates : (384, 318) connected to slice MIoT @ BS0
Client_791, coordinates : (384, 318) requests 7320371 usage to BS0 to slice MIoT
Client 93, coordinates : (348, 495) connected to slice URLLC @ BS0
Client_93, coordinates : (348, 495

Client 776, coordinates: (550,292) disconnected from slice MIoT @ BS0 
Client 500, coordinates: (551.8589563766718,51.315409397501085) disconnected from slice MIoT @ BS0 
STEP 1
Client 549, coordinates: (467, 446) gets 100000000 usage from @ BS0 for slice eMBB
Client 12, coordinates: (72, 231) gets 594498 usage from @ BS0 for slice URLLC
Client 468, coordinates: (198.07550209874327, 386.1570227409356) gets 100000000 usage from @ BS0 for slice eMBB
Client 822, coordinates: (412.08595292244104, 380.53980136435115) gets 100000000 usage from @ BS0 for slice eMBB
Client 783, coordinates: (275, 528) gets 163400 usage from @ BS0 for slice MIoT
Client 791, coordinates: (383.9008982036429, 318.01912350925323) gets 163400 usage from @ BS0 for slice MIoT
Client 93, coordinates: (347.8506667427019, 494.97537162845487) gets 594498 usage from @ BS0 for slice URLLC
Client 554, coordinates: (434, 338) gets 100000000 usage from @ BS0 for slice eMBB
Client 806, coordinates: (222.25554542668877, 95.39730

Client 8, coordinates: (162, 532) gets 3089645 usage from @ BS0 for slice MIoT
Client 557, coordinates: (251.0308643656766, 449.1868317780304) gets 3207987 usage from @ BS0 for slice URLLC
Client 935, coordinates: (548, 73) gets 4144169 usage from @ BS0 for slice MIoT
Client 236, coordinates: (477, 413) gets 100000000 usage from @ BS0 for slice eMBB
Client 426, coordinates: (501.8550125648623, 531.0408232249902) gets 3207987 usage from @ BS0 for slice URLLC
Client 313, coordinates: (70, 117) gets 1000000 usage from @ BS0 for slice voice
Client 725, coordinates: (228.08672987947438, 211.95745262125973) gets 4034296 usage from @ BS0 for slice MIoT
Client 854, coordinates: (189, 136) gets 3293094 usage from @ BS0 for slice mMTC
Client 726, coordinates: (245.0339742356446, 511.1216502619899) gets 3207987 usage from @ BS0 for slice URLLC
Client 127, coordinates: (427.00033797168106, 327.0899848809937) gets 3063111 usage from @ BS0 for slice mMTC
Client 606, coordinates: (407, 459) gets 1000

Client 210, coordinates: (50,520) disconnected from slice eMBB @ BS0 
Client 445, coordinates: (247,550) disconnected from slice URLLC @ BS0 
Client 305, coordinates: (193,551) disconnected from slice MIoT @ BS0 
Client 98, coordinates: (440,50) disconnected from slice MIoT @ BS0 
Client 895, coordinates: (50,69) disconnected from slice mMTC @ BS0 
Client 776, coordinates: (550,291) disconnected from slice MIoT @ BS0 
STEP 3
Client 549, coordinates: (468, 447) gets 100000000 usage from @ BS0 for slice eMBB
Client 12, coordinates: (76, 227) gets 2074043 usage from @ BS0 for slice URLLC
Client 468, coordinates: (197.8553108460703, 385.98709829916976) gets 25752028 usage from @ BS0 for slice eMBB
Client 822, coordinates: (429.4790390644711, 376.8338146820709) gets 100000000 usage from @ BS0 for slice eMBB
Client 791, coordinates: (383.7538868608864, 317.96412038709434) gets 2659080 usage from @ BS0 for slice MIoT
Client 93, coordinates: (347.7909918353171, 495.2101823782346) gets 3103113 

Client 36, coordinates: (161,551) disconnected from slice eMBB @ BS0 
Client 500, coordinates: (553.7478072110474,62.319255938258316) disconnected from slice MIoT @ BS0 
Client 99, coordinates: (46.00026296618117,77.8519840100337) disconnected from slice eMBB @ BS0 
STEP 4
Client 549, coordinates: (469, 448) gets 58079688 usage from @ BS0 for slice eMBB
Client 468, coordinates : (197.8553108460703, 385.98709829916976) connected to slice eMBB @ BS0
Client_468, coordinates : (197.8553108460703, 385.98709829916976) requests 308092602 usage to BS0 to slice eMBB
Client 822, coordinates: (439.9454581960748, 365.7196011392763) gets 83250040 usage from @ BS0 for slice eMBB
Client 791, coordinates: (383.70309201358384, 318.00256758611386) gets 353722 usage from @ BS0 for slice MIoT
Client 554, coordinates: (434, 338) gets 96502854 usage from @ BS0 for slice eMBB
Client 501, coordinates : (517, 223) connected to slice mMTC @ BS0
Client_501, coordinates : (517, 223) requests 4625303 usage to BS0 

...

Client 678, coordinates: (97,560) disconnected from slice voice @ BS0 
Client 985, coordinates: (403,556) disconnected from slice voice @ BS0 
Client 445, coordinates: (251,557) disconnected from slice URLLC @ BS0 
Client 293, coordinates: (202,551) disconnected from slice eMBB @ BS0 
Client 435, coordinates: (188.94679885419632,46.43745233483619) disconnected from slice MIoT @ BS0 
Client 172, coordinates: (554,460) disconnected from slice eMBB @ BS0 
Client 98, coordinates: (441,46) disconnected from slice MIoT @ BS0 
Client 591, coordinates: (48,62) disconnected from slice voice @ BS0 
Client 477, coordinates: (71,551) disconnected from slice voice @ BS0 
Client 274, coordinates: (542,50) disconnected from slice voice @ BS0 
Client 678, coordinates: (97,558) disconnected from slice voice @ BS0 
Client 11, coordinates: (476.9133831231557,38.38049185785081) disconnected from slice voice @ BS0 
STEP 26
Client 544, coordinates: (201, 537) gets 347426 usage from @ BS0 for slice voice
Cli

Client 431, coordinates: (341,49) disconnected from slice MIoT @ BS0 
Client 272, coordinates: (554,474) disconnected from slice mMTC @ BS0 
Client 23, coordinates: (245,46) disconnected from slice mMTC @ BS0 
Client 537, coordinates: (550.0182010357764,323.1321202342844) disconnected from slice mMTC @ BS0 
Client 98, coordinates: (441,46) disconnected from slice MIoT @ BS0 
STEP 27
Client 951, coordinates: (359, 311) gets 1000000 usage from @ BS0 for slice voice
Client 916, coordinates: (444, 313) gets 1000000 usage from @ BS0 for slice voice
Client 959, coordinates: (247.64842046741654, 130.1101520513157) gets 1000000 usage from @ BS0 for slice voice
Client 75, coordinates: (317, 80) gets 1000000 usage from @ BS0 for slice voice
Client 300, coordinates: (340, 263) gets 1000000 usage from @ BS0 for slice voice
Client 988, coordinates: (537, 66) gets 1000000 usage from @ BS0 for slice voice
Client 448, coordinates: (400, 244) gets 720771 usage from @ BS0 for slice voice
Client 625, coo

Client 962, coordinates: (220.75321787438426, 55.17448287680022), releases 100000000 usage to BS0
Client 227, coordinates: (447.7442953179603, 344.20742572989667), releases 100000000 usage to BS0
Client 707, coordinates: (443.25898290326757, 340.7881833671708), releases 100000000 usage to BS0
Client 520, coordinates: (479, 373), releases 2287759 usage to BS0
Client 520, coordinates: (479,373) disconnected from slice mMTC @ BS0 
Client 969, coordinates: (116, 158), releases 1000000 usage to BS0
Client 81, coordinates: (460.4331218197247, 382.67850036985294), releases 1000000 usage to BS0
Client 868, coordinates: (327, 92), releases 1862274 usage to BS0
Client 868, coordinates: (327,92) disconnected from slice MIoT @ BS0 
Client 176, coordinates: (245, 388), releases 7198912 usage to BS0
Client 176, coordinates: (245,388) disconnected from slice URLLC @ BS0 
Client 901, coordinates: (432, 214), releases 100000000 usage to BS0
Client 112, coordinates: (212, 466), releases 100000000 usage 

Client 348, coordinates: (351, 91), releases 1000000 usage to BS0
Client 80, coordinates: (521, 85), releases 1000000 usage to BS0
Client 354, coordinates: (291, 470), releases 3058162 usage to BS0
Client 249, coordinates: (130, 57), releases 1000000 usage to BS0
Client 761, coordinates: (182, 306), releases 82024164 usage to BS0
Client 761, coordinates: (182,306) disconnected from slice eMBB @ BS0 
Client 493, coordinates: (310, 541), releases 1824523 usage to BS0
Client 252, coordinates: (463, 529), releases 6446437 usage to BS0
Client 252, coordinates: (463,529) disconnected from slice URLLC @ BS0 
Client 218, coordinates: (431, 92), releases 1000000 usage to BS0
Client 539, coordinates: (466, 504), releases 1000000 usage to BS0
Client 156, coordinates: (107, 385), releases 1000000 usage to BS0
Client 649, coordinates: (411.950497155501, 354.56513553359554), releases 100000000 usage to BS0
Client 549, coordinates: (480, 454), releases 67652028 usage to BS0
Client 549, coordinates: (

Client 988, coordinates: (539, 68), releases 1000000 usage to BS0
Client 762, coordinates: (359, 175), releases 175551 usage to BS0
Client 762, coordinates: (359,175) disconnected from slice voice @ BS0 
Client 865, coordinates: (330.24395946137264, 525.9636439185293), releases 18849451 usage to BS0
Client 865, coordinates: (330.24395946137264,525.9636439185293) disconnected from slice eMBB @ BS0 
Client 996, coordinates: (477, 359), releases 1000000 usage to BS0
Client 167, coordinates: (264, 379), releases 90536746 usage to BS0
Client 167, coordinates: (264,379) disconnected from slice eMBB @ BS0 
Client 201, coordinates: (285, 127), releases 1000000 usage to BS0
Client 798, coordinates: (146, 311), releases 1000000 usage to BS0
Client 586, coordinates: (465, 507), releases 1000000 usage to BS0
Client 542, coordinates: (358.3321370401959, 497.6342560644187), releases 1000000 usage to BS0
Client 257, coordinates: (463.31169092687685, 272.8534565136432), releases 982233 usage to BS0
Cl

In [10]:
with open('new_data.csv', mode='w') as csv_file:
    fieldnames = ['embb_rate', 'embb_users', 'embb_req', 'urllc_rate', 'urllc_users', 'urllc_req', 'miot_rate', 'miot_users', 'miot_req', 'mmtc_rate', 'mmtc_users', 'mmtc_req', 'voice_rate', 'voice_users', 'voice_req']
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
    
    writer.writeheader()
    for item in csv_final:
        writer.writerow({'embb_rate': item.g_embb, 'embb_users': item.connect_embb, 'embb_req': item.r_embb, 'urllc_rate': item.g_urllc, 'urllc_users': item.connect_urllc, 'urllc_req': item.r_urllc, 'miot_rate': item.g_miot, 'miot_users': item.connect_miot, 'miot_req': item.r_miot, 'mmtc_rate': item.g_mmtc, 'mmtc_users': item.connect_mmtc, 'mmtc_req': item.r_mmtc, 'voice_rate': item.g_voice, 'voice_users': item.connect_voice, 'voice_req': item.r_voice})


In [11]:
for item in csv_final:
    print(item.r_urllc, item.connect_urllc)

0 0
433200828 68
387563286 67
172628157 67
15899367 21
138493115 23
155345124 29
161910816 38
147635014 35
102152151 28
150629963 28
111564779 24
141695157 27
86019738 20
211056979 35
122807609 34
81558722 24
147074775 28
109893368 23
166747292 30
171934179 34
88813920 27
189456303 34
150210091 36
103372971 29
170304065 35
66059308 21
175715894 30
91750979 23
126567120 25
